In [ ]:
#import all the required libraries

#make sure all these libraries are installed 

!conda install -c conda-forge selenium --yes # Install selenium for conda
!conda install -c anaconda beautifulsoup4 --yes #Install beautifulsoup
!conda install -c anaconda lxml --yes
!conda install -c anaconda html5lib --yes
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c anaconda wget --yes


import numpy as np
import urllib.request
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import folium # map rendering library
print("All packages have been imported")
#There are Multiple parts to this assignment..
#1st is web scraping

Solving environment: | 
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0
  - defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35ccdone

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         156 KB  anaconda
    ------------------------------------------------------------
                                           Total:         156 KB

The following packages will be SUPERSEDED by a higher-priority channel:

  

In [ ]:
# URL for the wikipedia page is given below
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# assign a variabl to the page or import the page into a variable
page = urllib.request.urlopen(wiki_url)

In [ ]:
#convert the page (HTML) into beautiful soupe format
soup = BeautifulSoup(page, "html5lib")
# soup
# print(soup.prettify())

In [ ]:
#class="wikitable sortable jquery-tablesorter"
all_tables=soup.find_all("table")
# all_tables
right_table=soup.find('table', class_="wikitable sortable")
# right_table


In [ ]:
# assigning variables which will be used for loading values in dataframe which will be created later
a,b,c=[],[],[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        a.append(cells[0].find(text=True))
        b.append(cells[1].find(text=True))
        c.append(cells[2].find(text=True))

        

# ##uncomment this in order to add a empty neigbourhood with some boruough for validaitng point 4
# a.append('V12B')
# b.append('test')
# c.append('\n')

In [ ]:
df=pd.DataFrame(a,columns=['Postal Code'])
df['Borough']=b
df['Neighbourhood']=c
# Replacing \n artifact that we got during creating table from soup
df = df.replace('\n',' ', regex=True)

In [ ]:
df_test = df
# removing the empty rows
# df_test=df_test[df_test['Neighbourhood'].str.strip().astype(bool)] #

df_test = df_test[~df_test.Borough.str.contains("Not assigned")]

# checking to see if there are any couloums that are empty
# Empty_coloumn_test = df_test[df_test['Borough'].str.contains("Not assigned")] #Not assigned

#Reseting the index in order to start indexing from 0
# df_test.reset_index(inplace=True)
# Removing the extra index column that we got after using reset_index object
# df_test.drop(columns=['index'],inplace=True)
# appending all the strings together which we get by groupby
df_test = df_test.groupby(['Postal Code', 'Borough'], as_index=False).agg({'Neighbourhood': ' '.join})

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

# replace the empty Neighbourhood with borough
df_test['Neighbourhood'] = np.where((df_test['Neighbourhood'].str.strip().astype(bool) == False), df_test['Borough'], df_test['Neighbourhood'])


df_test


In [ ]:
df_test.shape
##comments lines where used for testing
# df_test1 = df_test
# df_test1[df_test1['Postal Code'].str.contains("M9V")]
# df_test[df_test['Neighbourhood'].str.strip().astype(bool)]